In [70]:
import pennylane as qml
from pennylane import numpy as np
from functools import partial
import matplotlib.pyplot as plt
from qubit_models import *
from helpers import *
import warnings
warnings.filterwarnings("ignore")

In [87]:
from datasets import *

train_X, test_X, train_y, test_y = get_xor_data(500)
# train_X, test_X, train_y, test_y = get_circular_boundary_dataset(500)
# train_X, test_X, train_y, test_y = get_moon_dataset(500)
len(train_X)

450

In [88]:
lambdas = {}
for i in range(8):
    lambdas[i+1] = qml.matrix(qml.GellMann(0,i+1))

In [89]:
s_params_size = 2
dev = qml.device("default.qutrit", wires=1)

def encode_and_rotate(x_i,s_params,w_params):
    U = (x_i[0]*s_params[0]*lambdas[6])+(x_i[1]*s_params[1]*lambdas[7])
    encode_U = np.exp(1j*U)
    qml.QutritUnitary(encode_U,wires=0)
    
    U = (w_params[0]*lambdas[1])+(w_params[1]*lambdas[4])
    rotate_U = np.exp(1j*U)
    qml.QutritUnitary(encode_U,wires=0)
    

@qml.qnode(dev)
def vqc_model(x_i, params):
    s_params,w_params = params[:s_params_size], params[s_params_size:]
    encode_and_rotate(x_i, s_params,w_params)
    obs = qml.GellMann(0,3)+np.sqrt(3)*qml.GellMann(0,8)
    return qml.expval(obs)

In [90]:
def loss(data, labels, model, params, binary=True):    
    loss_sum = []
    for idx in range(len(data)):
        data_point = data[idx]
        true_label = labels[idx]
        model_output = model(data_point, params)
        if binary == True:
            if (model_output<0 and true_label>0) or (model_output>0 and true_label<0):
                loss_sum.append((model_output - true_label) ** 2)
        else:
            model_class = 0
            if -2 <= model_output and model_output < (-2/3):
                model_class = -2
            elif (2/3) <= model_output and model_output <= 2:
                model_class = 2

            if model_class != true_label:
                loss_sum.append((model_output - true_label) ** 2)

    return sum(loss_sum)/len(data)

def make_prediction(model, data_point, params, binary=True):
    model_output = model(data_point, params)
    if binary == True:
        if model_output < 0:
            return -1
        return 1
    else:
        if -2 <= model_output and model_output < (-2/3):
            return -2
        elif (2/3) <= model_output and model_output <= 2:
            return 2
        return 0

def compute_accuracy(data, labels, model, params):
    n_samples = len(data)
#     for x in range(n_samples):
#         print(make_prediction(model, data[x], params),labels[x])
    return np.sum(
        [make_prediction(model, data[x], params) == labels[x] for x in range(n_samples)
    ]) / n_samples

In [91]:
s_params_size, w_params_size = 2,2
params = np.random.normal(size=(s_params_size+w_params_size))#*100
print(params)
# opt = qml.AdamOptimizer(stepsize=0.00087)
opt = qml.GradientDescentOptimizer(stepsize=0.009)
num_its = 20
loss_over_time = []
# curr_train_X, curr_train_y = get_initial_dataset(train_X,train_y)
for itr in range(num_its):
    (_, _, _, params), _loss = opt.step_and_cost(loss, train_X, train_y, vqc_model, params)
    loss_over_time.append(_loss)
    print("Iteration:",itr, "Loss:",_loss)


[-0.54834155  0.20039513 -0.36981235  0.07251855]
Iteration: 0 Loss: 6.270039618796572
Iteration: 1 Loss: 4.606778995561707
Iteration: 2 Loss: 2.84565450323314
Iteration: 3 Loss: 2.482226719958454
Iteration: 4 Loss: 2.2430053684660614
Iteration: 5 Loss: 2.3434640087561447
Iteration: 6 Loss: 2.4270026320043447
Iteration: 7 Loss: 2.7548960331188868
Iteration: 8 Loss: 2.7789293541300544
Iteration: 9 Loss: 3.196501243585918
Iteration: 10 Loss: 3.136518486099104
Iteration: 11 Loss: 3.6792829228842963
Iteration: 12 Loss: 3.544103020297975
Iteration: 13 Loss: 4.173306892492775
Iteration: 14 Loss: 3.9618633072008125
Iteration: 15 Loss: 4.7285194854331305
Iteration: 16 Loss: 4.498282299295263
Iteration: 17 Loss: 5.483834590522862
Iteration: 18 Loss: 5.314355847954946
Iteration: 19 Loss: 6.853834426626272


In [95]:
training_accuracy = compute_accuracy(train_X, train_y, vqc_model, params)
testing_accuracy = compute_accuracy(test_X, test_y, vqc_model, params)

print(f"Training accuracy = {training_accuracy}")
print(f"Testing accuracy = {testing_accuracy}")

Training accuracy = 0.5533333333333333
Testing accuracy = 0.58


In [124]:
# train_X, test_X, train_y, test_y = get_wine_dataset()
train_X, test_X, train_y, test_y = get_xor_data(100)


In [125]:
for x,y in zip(train_X,train_y):
    print(x,y)
    if x[0]>0 and x[1]>0:
        y=2
    elif x[0]<0 and x[1]<0:
        y=-2
    else:
        y=0
    print("New:",x,y)

[1.23029068 1.20237985] -1
New: [1.23029068 1.20237985] 2
[-0.67246045 -0.35955316] -1
New: [-0.67246045 -0.35955316] -2
[-0.91282223  1.11701629] 1
New: [-0.91282223  1.11701629] 0
[-1.63019835  0.46278226] 1
New: [-1.63019835  0.46278226] 0
[-0.35399391 -1.37495129] -1
New: [-0.35399391 -1.37495129] -2
[-0.10321885  0.4105985 ] 1
New: [-0.10321885  0.4105985 ] 0
[-0.74475482 -0.82643854] -1
New: [-0.74475482 -0.82643854] -2
[-0.09845252 -0.66347829] -1
New: [-0.09845252 -0.66347829] -2
[ 2.26975462 -1.45436567] 1
New: [ 2.26975462 -1.45436567] 0
[-0.81314628 -1.7262826 ] -1
New: [-0.81314628 -1.7262826 ] -2
[1.86755896 0.90604466] -1
New: [1.86755896 0.90604466] 2
[-0.80340966 -0.68954978] -1
New: [-0.80340966 -0.68954978] -2
[-0.51080514 -1.18063218] -1
New: [-0.51080514 -1.18063218] -2
[1.48825219 1.89588918] -1
New: [1.48825219 1.89588918] 2
[ 0.39600671 -1.09306151] 1
New: [ 0.39600671 -1.09306151] 0
[0.15494743 0.37816252] -1
New: [0.15494743 0.37816252] 2
[-1.16514984  0.900826